# Using Fourier Transform for Blurring, Sharpening
stough 202-
DIP 4.7 shows similar.

We saw in [`fft_intro`](./fft_intro.ipynb) that we can denoise an image in the Fourier space, also called the frequency domain. We also saw that we can consider the magnitude and phase components separately given the Fourier Transform of an image. Here we're going to play around further in the Fourier domain and show how we can blur and sharpen an image through clever manipulation in the freqency domain.

## Imports
We'll use the [Fast Fourier Functions](https://docs.scipy.org/doc/scipy/tutorial/fft.html) again.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage.color import rgb2gray
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_image,
                       vis_pair,
                       vis_surface)

from scipy.fft import (fft2, ifft2, fftshift, 
                       ifftshift, fftn, ifftn)
from skimage.transform import resize

## Blur, Edginess, Sharpen
In this worksheet we'll take an image and manipulate it in the frequency space.

- Use `fftn` to convert to the Fourier domain
- Scale and manipulate the spectrum coefficients in the Fourier domain.
- Use `ifftn` to convert back to the image/spatial domain.

Of note in this exercise is that visualization can be a bear. We'll use `fftshift` to visualize the Fourier domain in a way that shows the low-frequency components in the center of the image and higher frequencies out towards the extremes.

In [2]:
I = plt.imread('../dip_pics/switzerland_hazy.jpg')
I = I/255.0
arr_info(I)

((1536, 2048, 3), dtype('float64'), 0.0, 1.0)

In [3]:
vis_image(I, show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
I_fft = fftn(I)

In [5]:
arr_info(I_fft)

((1536, 2048, 3),
 dtype('complex128'),
 (-231818.46735175475-282717.4222377647j),
 (4825182.023529412-0j))

### Visualizing in the Fourier domain
A bit difficult artifact about `fft` is that the output places the lower-frequency coefficients at the corners of the transform array. Normally however, we wish to visualize the spectrum such that lower frequencies are in the center of the figure. We'll use `fftshift` to do this.

In [6]:
# Visualize the freq domain.
temp = np.abs(I_fft)
temp = np.log2(1 + temp)/np.log2(1+temp.max())
vis_image(fftshift(temp))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Using a Gaussian for smooth scaling

In [`fft_intro`](./fft_intro.ipynb) we simply zero'd out all the coefficients outside of some square in the fourier space in order to eliminate high-frequency content. However this can lead to weird artifacts that are related to [aliasing](https://en.wikipedia.org/wiki/Aliasing). The proper way to manipulate the fourier domain is through smoother scaling, like the Gaussian. Below we'll define a function that generates a 2D Gaussian of the same shape as an input image. We did something like this using scipy's `norm` and `np.outer` in [color_quantization](../SensingSamplingQuantization/color_quantization.ipynb). Here, we give ourselves a little more flexibility by directly evaluating the Gaussian exponential at all the pixel coordinates we want.

\begin{equation}
\mathcal{N}(\mu, \sigma)(x) = e^{-(x-\mu)^2/\sigma^2}
\end{equation}


In [7]:
# Thanks https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy

def gaussian(I, sig):
    x, y = np.meshgrid(np.arange(I.shape[1]), np.arange(I.shape[0]))
    coords = np.stack([x,y], axis=-1)
    c = np.reshape([I.shape[1]/2, I.shape[0]/2], (1,1,2))
    return np.exp((-(np.linalg.norm(coords-c, axis=2)**2))/(2*sig**2))

In [8]:
vis_image(gaussian(I, 200))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Blur: Use the Gaussian as a scalar mutliple in the frequency domain
Great reference for [matplotlib surface visualizations](https://matplotlib.org/stable/gallery/index.html#mplot3d-examples-index). Here I just want to visualize what the scalar we'll use looks like over the image, as a surface in 3D.

In [9]:
vis_surface(gaussian(I, 200))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### The Actual scaling in frequency space
We want to multiply the Fourier coefficients of the image by this Gaussian image, thereby scaling the coefficients. We just have to remember that `I_fft` needs to be `fftshift`'d to make the pixels line up correctly. We only do the shifting for visualization until now. Note that the variable `temp` comes from several cells ago, when we first visualized `I_fft`. 

In [10]:
vis_pair(fftshift(temp), gaussian(I, 100), 
         first_title='Fourier Spectrum of Image', 
         second_title='Gaussian Scaling Factor',
         show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
blur_I_fft = ifftshift(np.multiply(fftshift(I_fft), gaussian(I, 100)[...,None]))

In [12]:
# Visualize the freq domain.
temp = np.abs(blur_I_fft)
temp = np.log2(1 + temp)/np.log2(1+temp.max())
vis_image(fftshift(temp))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
I_blurred = ifftn(blur_I_fft)
arr_info(I_blurred)

((1536, 2048, 3),
 dtype('complex128'),
 (0.07312906139033189-1.4230526765533602e-17j),
 (0.9999999994021648+5.3553954578962766e-17j))

In [14]:
vis_pair(I, np.clip(I_blurred.real, 0, 1), second_title='Blurred in Freq Space', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Edge Finder: Use the complement of the Gaussian as the scalar
Thereby keeping only the higher frequency content.

In [24]:
vis_surface(1-gaussian(I, 200))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
edge_I_fft = ifftshift(np.multiply(fftshift(I_fft), 1-gaussian(I, 100)[...,None]))

In [17]:
# Visualize the freq domain.
temp = np.abs(edge_I_fft)
temp = np.log2(1 + temp)/np.log2(1+temp.max())
vis_image(fftshift(temp))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
I_edges = ifftn(edge_I_fft)
arr_info(I_edges.real)

((1536, 2048, 3), dtype('float64'), -0.5746784645962378, 0.6157075032323429)

In [19]:
temp = np.abs(I_edges.real)
temp = (temp - temp.min())/(temp.max()-temp.min()) # Normalize - compress the range of max and min to 1 to 0
arr_info(temp)

((1536, 2048, 3), dtype('float64'), 0.0, 1.0)

In [20]:
arr_info(I+temp.max()-temp)

((1536, 2048, 3), dtype('float64'), 0.08232349828984364, 2.0)

In [21]:
vis_pair(I, temp.max()-temp, second_title='Just High Freq', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## How might we potentially Sharpen the image?

In [28]:
np.array([1,2,3])[...,None].shape

(3, 1)

In [25]:
x = (1-gaussian(I,200))
y = 1 + 4*x
edge_I_fft2 = ifftshift(np.multiply(fftshift(I_fft), y[...,None]))

In [22]:
edge_I_fft2 = ifftshift(np.multiply(fftshift(I_fft), 2-gaussian(I, 100)[...,None]))

In [29]:
arr_info(edge_I_fft2)

((1536, 2048, 3),
 dtype('complex128'),
 (-231830.05820267939-282731.55802052777j),
 (4825182.023529412+0j))

In [26]:
I_edges2 = ifftn(edge_I_fft2)
temp2 = np.abs(I_edges2.real)
temp2 = np.clip(temp2,0,1)
vis_pair(I, temp2, second_title='Edge Sharpen', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …